# Smart Water Quality Monitoring System - Deployment
    
1. [Classification Model Deployment](#Classification-Model-Deployment)
    -  [1.1 Dataset Preparation](#DataSet-Preparation)
    -  [1.2 Prepare Model](#Prepare-Model)
    -  [1.3 Deploy Model](#Deploy-Model)
    -  [1.4 Test Model](#Test-Model)
    
2. [Anomaly Detection Model Deployment](#AnomalyDetection-Model-Deployment)
    -  [2.1 Dataset Preparation](#DataSet-Preparation2)
    -  [2.2 Prepare Model](#Prepare-Model2)
    -  [2.3 Deploy Model](#Deploy-Model2)
    -  [2.4 Test Model](#Test-Model2)

## 1. Classification Model Deployment <a class="anchor" id="Classification-Model-Deployment"></a>

In [6]:
#Suppress warning messages

import sys
import time
import warnings
from IPython.display import Markdown, display

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [7]:
from sklearn.ensemble import RandomForestClassifier
from azureml import Workspace
from azureml import services
from sklearn.cross_validation import train_test_split
from azure.storage.blob import BlockBlobService
import pandas as pd
import numpy as np

### 1.1. Dataset Preparation <a class="anchor" id="DataSet-Preparation"></a>

In [11]:
#LOCALFILE is the file path    
STORAGEACCOUNTNAME= "smartwaterstorage"
STORAGEACCOUNTKEY= "5GGqEdDGjlqyKRjnLIFogG1LzeUlyQ3iwMBFyDgR7pdB4VPMtuUwvBNUyAss98wrNTWa8KNTe+L7DxMcEWyY5Q=="
LOCALFILENAME= "SW"    
CONTAINERNAME= "smart-water-db"
blob_service=BlockBlobService (account_name=STORAGEACCOUNTNAME,account_key=STORAGEACCOUNTKEY)

BLOBNAME= "ModelTraining/Dataset/SW_TrainingData.csv"
blob_service.get_blob_to_path(CONTAINERNAME,BLOBNAME,LOCALFILENAME)   

# Read Blob storage data
df_all = pd.read_csv(LOCALFILENAME, header=0) 
    
# Standardize the labels generated by the models
array = df_all.values
X = array[:,4:9]
Y = array[:,9]
Y = Y.astype('int')

validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split (X, Y, test_size=validation_size, random_state=seed)

### 1.2. Prepare Model <a class="anchor" id="Prepare-Model"></a>

In [12]:
model = RandomForestClassifier(min_samples_split = 2, 
                               bootstrap = True, 
                               min_samples_leaf = 1, 
                               n_estimators = 100, 
                               max_features = 4, 
                               max_depth = 10)
model.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=10, max_features=4,
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0)

### 1.3. Deploy Model <a class="anchor" id="Deploy-Model"></a>

In [13]:
# extract workspace info
ws = Workspace("9240b66313e840449c47cb8b1ed98a93", "9uPJO9gPZAM3LE2Z074MRqg+zTIgKdYf7qLgtXote45GCpg/e2AWdPnFbTLlGTpDMKsjHa3A7Svz0/jH2ILxig==")
workdspace_id = ws.workspace_id
authorization_token = ws.authorization_token

# set up web service
from azureml import services
@services.publish(workdspace_id, authorization_token)
@services.types(WT=float, PH=float, DO=float, ORP=float, COND=float)
@services.returns(int)
def SmartWater_MultiClassPredictor_2(WT, PH, DO, ORP, COND):
    # predict the label
    feature_vector = [WT, PH, DO, ORP, COND]
    return model.predict(feature_vector)

# save information about the web service
service_url = SmartWater_MultiClassPredictor_2.service.url 
api_key = SmartWater_MultiClassPredictor_2.service.api_key
help_url = SmartWater_MultiClassPredictor_2.service.help_url
service_id = SmartWater_MultiClassPredictor_2.service.service_id

### 1.4. Test Model <a class="anchor" id="Test-Model"></a>

In [15]:
print(SmartWater_MultiClassPredictor_2(24.21,6.15,67.3,-0.057,517.2))

[3]


## 2. Anomaly Detection Model Deployment <a class="anchor" id="AnomalyDetection-Model-Deployment"></a>

In [9]:
from sklearn import svm

### 2.1. Dataset Preparation <a class="anchor" id="DataSet-Preparation2"></a>

In [10]:
#LOCALFILE is the file path    
STORAGEACCOUNTNAME= "smartwaterstorage"
STORAGEACCOUNTKEY= "5GGqEdDGjlqyKRjnLIFogG1LzeUlyQ3iwMBFyDgR7pdB4VPMtuUwvBNUyAss98wrNTWa8KNTe+L7DxMcEWyY5Q=="
LOCALFILENAME= "SW"    
CONTAINERNAME= "smart-water-db"
blob_service=BlockBlobService (account_name=STORAGEACCOUNTNAME,account_key=STORAGEACCOUNTKEY)
    
BLOBNAME= "ModelTraining/Dataset/Accelerometer_Test.csv"
blob_service.get_blob_to_path(CONTAINERNAME,BLOBNAME,LOCALFILENAME)   

# Read Blob storage data
df_Accelerometer = pd.read_csv(LOCALFILENAME, header=0) 

### 2.2. Prepare Model <a class="anchor" id="Prepare-Model2"></a>

In [11]:
X_train = df_Accelerometer[['ACC_X', 'ACC_Y', 'ACC_Z']].values

ACC_X = df_Accelerometer["ACC_X"].values
ACC_Y = df_Accelerometer["ACC_Y"].values
ACC_Z = df_Accelerometer["ACC_Z"].values
t =[x for x in range(len(ACC_X))]

model = svm.OneClassSVM(nu=0.0084, kernel="rbf", gamma=0.0001)

model.fit(X_train)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.0001, kernel='rbf',
      max_iter=-1, nu=0.0084, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

### 2.3. Deploy Model <a class="anchor" id="Deploy-Model2"></a>

In [12]:
# extract workspace info
ws = Workspace("9240b66313e840449c47cb8b1ed98a93", "9uPJO9gPZAM3LE2Z074MRqg+zTIgKdYf7qLgtXote45GCpg/e2AWdPnFbTLlGTpDMKsjHa3A7Svz0/jH2ILxig==")
workdspace_id = ws.workspace_id
authorization_token = ws.authorization_token

# set up web service
from azureml import services
@services.publish(workdspace_id, authorization_token)
@services.types(ACC_X=int, ACC_Y=int, ACC_Z=int)
@services.returns(int)
def SmartWater_AnomalyDetector_2(ACC_X, ACC_Y, ACC_Z):
    # predict the label
    feature_vector = [ACC_X, ACC_Y, ACC_Z]
    return model.predict(feature_vector)

# save information about the web service
service_url = SmartWater_AnomalyDetector_2.service.url 
api_key = SmartWater_AnomalyDetector_2.service.api_key
help_url = SmartWater_AnomalyDetector_2.service.help_url
service_id = SmartWater_AnomalyDetector_2.service.service_id

### 2.4. Test Model <a class="anchor" id="Test-Model2"></a>

In [15]:
print(SmartWater_AnomalyDetector_2(-6,100,1026))

[-1.]
